**INSTALLING LIBRARIES**

In [13]:
import pandas as pd
import cv2
import sklearn
import dlib
import imutils
import playsound     #plays sound with single line of code
from imutils.video import VideoStream
import os
import time
from scipy.spatial import distance as dist
from pygame import mixer  #used for loading and playing sounds

**CREATING REQUIRED FUNCTIONS**

In [14]:
#function for playing sound
def play_sound(path):
  playsound.playsound(path)


#function for finding eye aspect ratio
#defining euclidean distance between vertical eye landmarks
def eye_aspect_ratio(eye):
  A = dist.euclidean(eye[1],eye[5])
  B = dist.euclidean(eye[2],eye[4])

#defining euclidean distance between horizontal eye landmarks
  C = dist.euclidean(eye[0],eye[3])

#computing aspect ratio of eyes
  eye_asp_r = (A+B) / (2.0 * C)

  return eye_asp_r

**INTIALIZING PARAMETERS**

In [15]:
#setting font for the counter
font = cv2.FONT_HERSHEY_TRIPLEX

In [16]:
mixer.init() #starting the mixer
al_sound = mixer.Sound(r"C:\Users\Rohit\Desktop\\My DL Projects\Driver Drowsiness Detection\siren.WAV")

In [17]:
#defining threshold

#If aspect ratio falls below this threshold then the model will start counting 
#the number of frames the person has closed their eyes for
eye_thresh = 0.25

#if the number of frames having eyes closed exceeds this value , then alarm will be played
eye_frame_check = 40

#counting number of frames having aspect ratio less than eye_thresh
counter = 0

#if counter exceeds 40 then the boolean alarm_on will be updated
alarm_on = False

In [18]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:\\Users\Rohit\Desktop\\My DL Projects\Driver Drowsiness Detection\shape_predictor_68_face_landmarks.dat")

In [19]:
from imutils import face_utils
#the 68 landmarks are encoded inside FACIAL_LANDMARKS_IDXS
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"] #taking coordinates of left eye
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"] #taking coordinates of right eye

**CORE OF DROWSINESS DETECTOR**

In [20]:
#to get a video capture object for the camera.
vid = cv2.VideoCapture(0)

#checking whether video capturing is started or not
if vid.isOpened():
    vid = cv2.VideoCapture(0)
    print("yes :)")
if not vid.isOpened():
    raise IOError("Cannot open webcam :(")

yes :)


In [21]:
print("starting video stream thread!!")

cv2.namedWindow("Driver_Drowsiness") #window showing the output video

# loop over frames from the video stream
while (True):
    
    ret, frame = vid.read()    # grab and read the frames using the above created object
    height,width = frame.shape[:2]     # resizing the height to width ratio of frames
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    # converting the frames to grayscale
    # grayscale simplifies the algorithm and reduces computational requirements
    
    
    # detect faces in the grayscale frame
    rects = detector(gray, 0)  

    for rect in rects:  # looping over each grayscale frame
        
        shape = predictor(gray, rect)   # predicts 68 landmarks
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[lStart:lEnd]  # ratio of euclidean distances between landmarks of left eye
        rightEye = shape[rStart:rEnd]  #  ratio of euclidean distances between landmarks of right eye
        leftEAR = eye_aspect_ratio(leftEye)  # EAR of left eye
        rightEAR = eye_aspect_ratio(rightEye)  # EAR of right eye
   
        eye_asp_r = (leftEAR + rightEAR) / 2.0  # average of EAR of both eyes

    # convex hull of an object is the minimum boundary that can completely enclose or wrap the object(or contour of that object)
        
        leftEyeHull = cv2.convexHull(leftEye)  # convexHull of left eye
        rightEyeHull = cv2.convexHull(rightEye) # convexHull of right eye
        
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)  # drawing contours around left eye
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)  # #drawing contours around left eye

        
        if eye_asp_r < eye_thresh:  #counting number of frames when the value falls below threshold value
            counter += 1

            # if the eyes were closed for a sufficient number of frames then sound the alarm
            if counter >= eye_frame_check:
                
                if not alarm_on:     # if the alarm is not on, turn it on
                    al_sound.play()
                            
                    cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)  # draw an alarm notice on the frame
                    
        else:     # otherwise, the eye aspect ratio is not below the blink threshold, so reset the counter and alarm
            counter = 0
            alarm_on = False

        cv2.putText(frame, "EAR: {:.2f}".format(eye_asp_r), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)  # displaying EAR value on screen
 
# show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF   # waitKey(0) will pause your screen because it will wait infinitely for 
                                  # keyPress on your keyboard and will not refresh the frame(cap.read()) using 
                                  # your WebCam. waitKey(1) will wait for keyPress for just 1 millisecond and
                                  # it will continue to refresh and read frame from your webcam using cap.read().

        # if the `q` key is pressed, break from the loop
    if key == ord("q"):
        break
# do a bit of cleanup
cv2.destroyAllWindows()
vid.release()

starting video stream thread!!
